In [1]:
## import packages
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indic

In [2]:
# read in data
train = pd.read_csv('training_set.csv',encoding = 'unicode_escape', parse_dates=True)
#test = pd.read_csv('holdout_set.csv',encoding = 'unicode_escape', parse_dates=True)

In [3]:
# convert date to datetime
train['Created'] = pd.to_datetime(train['Created'])
train.dtypes

/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)
/anaconda3/lib/python3.6/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


Engagements                      int64
Followers at Posting             int64
Created                 datetime64[ns]
Type                            object
Description                     object
dtype: object

In [4]:
# create date based fields
train['weekday'] = train['Created'].apply(lambda x: x.weekday())
train['weekend'] = np.where(train['weekday'] >= 4,'Y','N')
train['hour'] = train['Created'].dt.hour
train['timeOfDay'] = np.where((train['hour'] >= 2) & (train['hour'] <=11) ,'Morning',
                              np.where((train['hour'] >= 12) & (train['hour'] <=15),'Afternoon', np.where(
                                  (train['hour'] >= 16) & (train['hour'] <=19),'Evening','Night')))
train['mon'] = train['Created'].dt.month
train['offseason'] = np.where((train['mon'] >= 7) & (train['mon'] <=9) ,'Y','N')

In [5]:
train.head()

,Engagements,Followers at Posting,Created,Type,Description,weekday,weekend,hour,timeOfDay,mon,offseason
0,502093,36984682,2019-05-21 23:30:51,Video,The @raptors bench trio of @sergeibaka @norman...,1,N,23,Night,5,N
1,603380,36984682,2019-05-21 22:53:33,Video,@kyle_lowry7 pulls from deep for the @raptors ...,1,N,22,Night,5,N
2,603380,36984682,2019-05-21 22:19:58,Video,@k_mid22 with some english on the @bucks dime!,1,N,22,Night,5,N
3,725100,36984682,2019-05-21 22:02:41,Video,Kawhi punches it home with the left on TNT!,1,N,22,Night,5,N
4,661446,36984682,2019-05-21 20:47:49,Video,@giannis_an34 goes baseline early to rock the ...,1,N,20,Night,5,N


In [6]:
# create lowercase caption for text based features
train['DescLower'] = train['Description'].str.lower()

In [7]:
## create caption based fields
train['CapLength'] = train['Description'].str.len().fillna(0) # length of caption
# list of most popular players based on jersey sales from 2018 and 2019, including instagram specific names
popPlayerRegex = 'lebron|kingjames|curry|stephencurry|giannis|kyrie|embiid|harden|westbrook|russwest44|durant|easymoneysniper|porzingis|kporzee|simmons|bensimmons|dwyane|kawhi'
train['popPlayer'] = train['DescLower'].str.contains(popPlayerRegex,regex=True)*1
# list of most popular teams based on juersey sales from 2018 and 2019
popTeamRegex = 'lakers|warriors|celtics|knicks|sixers|bucks|thunder|rockets|raptors|cavs'
train['popTeam'] = train['DescLower'].str.contains(popTeamRegex,regex=True)*1
# post is tagged at another insta handle
train['at'] = train['DescLower'].str.contains('@',regex=True)*1
# post contains a hashtag
train['hash'] = train['DescLower'].str.contains('#',regex=True)*1
# tries to capture games on national tv
natlTvRegex = 'tnt|espn|abc|nbatv'
train['natlTV'] = train['DescLower'].str.contains(natlTvRegex,regex=True)*1
# posts about an even such as allstar game or playoffs or draft
train['event'] = train['DescLower'].str.contains('allstar|all-star|finals|playoff|draft|slamdunk',regex=True)*1

In [8]:
train.head()

,Engagements,Followers at Posting,Created,Type,Description,weekday,weekend,hour,timeOfDay,mon,offseason,DescLower,CapLength,popPlayer,popTeam,at,hash,natlTV,event
0,502093,36984682,2019-05-21 23:30:51,Video,The @raptors bench trio of @sergeibaka @norman...,1,N,23,Night,5,N,the @raptors bench trio of @sergeibaka @norman...,95.0,0,1,1,0,0,0
1,603380,36984682,2019-05-21 22:53:33,Video,@kyle_lowry7 pulls from deep for the @raptors ...,1,N,22,Night,5,N,@kyle_lowry7 pulls from deep for the @raptors ...,64.0,0,1,1,0,1,0
2,603380,36984682,2019-05-21 22:19:58,Video,@k_mid22 with some english on the @bucks dime!,1,N,22,Night,5,N,@k_mid22 with some english on the @bucks dime!,46.0,0,1,1,0,0,0
3,725100,36984682,2019-05-21 22:02:41,Video,Kawhi punches it home with the left on TNT!,1,N,22,Night,5,N,kawhi punches it home with the left on tnt!,43.0,1,0,0,0,1,0
4,661446,36984682,2019-05-21 20:47:49,Video,@giannis_an34 goes baseline early to rock the ...,1,N,20,Night,5,N,@giannis_an34 goes baseline early to rock the ...,57.0,1,0,1,0,1,0


In [48]:
## Look at popular words in description
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
import math

[nltk_data] Downloading package punkt to /Users/ivanchen/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [46]:
def remove_special_char(s):

    try:
        rst = re.sub('_','',re.sub('[^\w\s]','',s))
        rst = re.sub('\s+',' ',rst)
    except:
        print(s)
        if math.isnan(float(s)):
            return ''
    return rst.strip()

def get_doc(sents):
    doc_info = []
    i = 0
    for sent in sents:
        i+=1
        cnt = count_words(sent)
        temp = {'doc_id':i,'doc_length':cnt}
        doc_info.append(temp)
    return doc_info

def count_words(sent):
    cnt = 0
    words = word_tokenize(sent)
    for word in words:
        cnt += 1
    return cnt

def create_freq_dict(sents):
    i = 0
    freqDict_list = []
    for sent in sents:
        i+=1
        freq_dict = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            if word in freq_dict:
                freq_dict[word]+=1
            else:
                freq_dict[word]=1
            tmp = {'doc_id':i, 'freq_dict':freq_dict}
        freqDict_list.append(tmp)
    return freqDict_list

def calTF(doc_info,freqDict_list):
    tf_scores = []
    for tmpdict in freqDict_list:
        myid = tmpdict['doc_id']
        for k in tmpdict['freq_dict']:
            tmp = {'doc_id':myid,
                  'TF_score': tmpdict['freq_dict'][k]/doc_info[myid-1]['doc_length'],
                  'key':k}
            tf_scores.append(tmp)
    return tf_scores

def calIDF(doc_info,freqDict_list):
    idf_scores = []
    cnt = 0
    for tmpdict in freqDict_list:
        cnt+=1
        for k in tmpdict['freq_dict'].keys():
            count = sum([k in tmpdict['freq_dict'] for tmpdict in freqDict_list])
            tmp = {'doc_id':cnt,
                  'IDF_score': math.log(len(doc_info)/count),
                  'key':k}
            idf_scores.append(tmp)
    return idf_scores

def calTFIDF (tf_scores,idf_scores):
    tfidf_Scores = []
    for j in idf_scores:
        for i in tf_scores:
            if j['key'] == i['key'] and j['doc_id'] == i['doc_id']:
                temp = {
                    'doc_id':j['doc_id'],
                    'tfidf_score': j['IDF_score']*i['TF_score'],
                    'key':i['key']
                }
                break
        tfidf_Scores.append(temp)
    return tfidf_Scores

In [49]:
post_ls = train.DescLower.values
post_ls = [remove_special_char(s) for s in post_ls]
doc_info = get_doc(post_ls)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [51]:
freqDict_list = create_freq_dict(post_ls)
tfidf_scores = calTFIDF(calTF(doc_info,freqDict_list),calIDF(doc_info,freqDict_list))


In [52]:
tfidf_scores


[{'doc_id': 1, 'tfidf_score': 0.028444010204052836, 'key': 'the'},
 {'doc_id': 1, 'tfidf_score': 0.26391573724190903, 'key': 'raptors'},
 {'doc_id': 1, 'tfidf_score': 0.40710099801929506, 'key': 'bench'},
 {'doc_id': 1, 'tfidf_score': 0.6398221793065437, 'key': 'trio'},
 {'doc_id': 1, 'tfidf_score': 0.137618073869678, 'key': 'of'},
 {'doc_id': 1, 'tfidf_score': 0.46854394553523154, 'key': 'sergeibaka'},
 {'doc_id': 1, 'tfidf_score': 0.49129064061512684, 'key': 'normanpowell4'},
 {'doc_id': 1, 'tfidf_score': 0.5008285972311642, 'key': 'fredvanvleet'},
 {'doc_id': 1, 'tfidf_score': 0.49129064061512684, 'key': 'combine'},
 {'doc_id': 1, 'tfidf_score': 0.08651498914715523, 'key': 'for'},
 {'doc_id': 1, 'tfidf_score': 0.5008285972311642, 'key': '48'},
 {'doc_id': 1, 'tfidf_score': 0.11270559697728996, 'key': 'in'},
 {'doc_id': 1, 'tfidf_score': 0.20003796605043264, 'key': 'game'},
 {'doc_id': 1, 'tfidf_score': 0.33336079065308727, 'key': '4'},
 {'doc_id': 2, 'tfidf_score': 0.427405759483543

In [55]:

sorted(tfidf_scores, key=lambda x: x['tfidf_score'],reverse = True)

[{'doc_id': 758, 'tfidf_score': 8.957510510291613, 'key': '1vs2'},
 {'doc_id': 783, 'tfidf_score': 8.957510510291613, 'key': 'trae'},
 {'doc_id': 857, 'tfidf_score': 8.957510510291613, 'key': 'domination'},
 {'doc_id': 1129, 'tfidf_score': 8.957510510291613, 'key': '0x0'},
 {'doc_id': 1145, 'tfidf_score': 8.957510510291613, 'key': '32294'},
 {'doc_id': 1536, 'tfidf_score': 8.957510510291613, 'key': 'remix'},
 {'doc_id': 1683, 'tfidf_score': 8.957510510291613, 'key': 'vooch'},
 {'doc_id': 2365, 'tfidf_score': 8.957510510291613, 'key': '23x23'},
 {'doc_id': 3345, 'tfidf_score': 8.957510510291613, 'key': 'therondo'},
 {'doc_id': 4468, 'tfidf_score': 8.957510510291613, 'key': 'dubs'},
 {'doc_id': 4474, 'tfidf_score': 8.957510510291613, 'key': 'ayyyyyyy'},
 {'doc_id': 4496, 'tfidf_score': 8.957510510291613, 'key': 'ahhhhhh'},
 {'doc_id': 5043, 'tfidf_score': 8.957510510291613, 'key': 'playoffrondo'},
 {'doc_id': 5196, 'tfidf_score': 8.957510510291613, 'key': 'rsherman25'},
 {'doc_id': 5815,

In [63]:
## get unique words
word_ls = [x['key'] for x in tfidf_scores]
score_ls = [x['tfidf_score'] for x in tfidf_scores]
word_set = set([x['key'] for x in tfidf_scores])
len(word_set)

5782

In [70]:
## get avg tfidf scores for each word
word_dict = {}
for word in word_set:
    indices = [i for i,x in enumerate(tfidf_scores) if x['key'] == word]
    scores = [score_ls[x] for x in indices]
    word_dict[word] = sum(scores)/len(scores)
word_dict

{'ipromiseschool': 0.2885548723037641,
 'app': 0.3165967409980807,
 '1967': 0.2985836836763871,
 '96rpg': 0.8143191372992375,
 'year': 0.6052771059700697,
 'decisionmaker': 1.2796443586130875,
 'usabwnt': 0.4071595686496188,
 'bango': 1.7915021020583226,
 '3tross1': 1.0066276016512403,
 'connection': 1.1860752571348556,
 'gorguidiengs': 1.2796443586130875,
 'denver': 0.5725856388115319,
 'dont': 0.9855307539574675,
 'felt': 0.6398221793065437,
 'rating': 0.5269123829583301,
 'proud': 0.6639145071111514,
 'jdg7873': 1.0643498227684722,
 'nbaenméxico': 1.002207980282795,
 'fraternity': 2.239377627572903,
 'dallasmavshoustonrockets': 0.8957510510291613,
 'surprise': 0.4572625495224134,
 'asked': 2.239377627572903,
 'bradbeal3s': 1.2052196522525347,
 'gtemp17': 0.7433633166340846,
 'wins': 0.521005756735348,
 'link': 0.39505401910868865,
 'watchmework': 0.456518673920786,
 'cj': 0.553417187258817,
 '33': 0.47663514754163616,
 'en': 0.5414462651589795,
 'ivicazubac': 1.0244221307001087,
 'j

In [71]:
sorted(word_dict.items(),key = lambda x: x[1],reverse = True)

[('playoffrondo', 8.957510510291613),
 ('domination', 8.957510510291613),
 ('rsherman25', 8.957510510291613),
 ('trae', 8.957510510291613),
 ('0x0', 8.957510510291613),
 ('ginobiliiiiii', 8.957510510291613),
 ('1vs2', 8.957510510291613),
 ('vooch', 8.957510510291613),
 ('bam', 8.957510510291613),
 ('fridaythe13th', 8.957510510291613),
 ('ayyyyyyy', 8.957510510291613),
 ('doubledunk', 8.957510510291613),
 ('remix', 8.957510510291613),
 ('therondo', 8.957510510291613),
 ('32294', 8.957510510291613),
 ('dubs', 8.957510510291613),
 ('bbqchickenalert', 8.957510510291613),
 ('ahhhhhh', 8.957510510291613),
 ('23x23', 8.957510510291613),
 ('dunkman', 7.858898221623503),
 ('sup', 5.509575553154445),
 ('ballgame', 5.239265481082335),
 ('grindcity', 5.165227081082292),
 ('reggiemillertnt', 4.958617997839),
 ('waiters3', 4.722493331275238),
 ('smoooooth', 4.478755255145806),
 ('suction', 4.478755255145806),
 ('ginobillllliiiiiii', 4.478755255145806),
 ('kat', 4.478755255145806),
 ('fear', 4.478755

In [78]:
## distribute engagement based on avg tfidf
word_engage = {}
for word in word_set:
    word_engage[word] = 0

for i,sent in enumerate(train.DescLower.values):
    this_words = word_tokenize(remove_special_char(sent))
    this_engage = train.Engagements.values[i]
    this_scores = []
    for word in this_words:
        if word in word_dict:
            this_scores.append(word_dict[word])
        else:
            this_scores.append(0)
    this_scores = np.array(this_scores) # tf idf values for each word
    this_scores = this_scores/sum(this_scores) # % of its tfidf values
    
    for j,word in enumerate(this_words):
        if word in word_engage:
            word_engage[word] += this_engage*this_scores[j]
            
    



nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [81]:
sorted(word_engage.items(),key = lambda x:x[1],reverse=True)

[('the', 61405203.52366523),
 ('for', 45341849.53704449),
 ('to', 39529813.78681452),
 ('in', 37728429.44991266),
 ('kingjames', 37382212.91725452),
 ('it', 34449052.74745733),
 ('with', 28854737.734769642),
 ('up', 27278453.305204228),
 ('on', 25173868.21858193),
 ('thisiswhyweplay', 24581250.009123094),
 ('of', 22656988.64735945),
 ('slam', 21870911.41364487),
 ('from', 20701102.05137535),
 ('nbaontnt', 19937731.026949573),
 ('nbapreseason', 19554866.220570587),
 ('nbaonabc', 19369848.820956256),
 ('nbaplayoffs', 19041929.77980225),
 ('stephencurry30', 18892793.52458515),
 ('a', 18841907.512596976),
 ('jharden13', 18836048.658198666),
 ('transition', 18711505.00964727),
 ('giannisan34', 18543649.860812314),
 ('action', 18517924.672773782),
 ('kyrieirving', 17771312.072909843),
 ('off', 17591915.7657081),
 ('x', 17522079.227170628),
 ('his', 17114334.039742626),
 ('back', 17102039.411699913),
 ('espn', 16880432.09214757),
 ('nbaallstar', 16429446.859087087),
 ('phantomcam', 16398255.8

In [77]:
a = np.array([1,2,3,4])
b = a/sum(a)


0.1

In [33]:
train.isnull().sum()

Engagements              0
Followers at Posting     0
Created                  0
Type                     0
Description             14
weekday                  0
weekend                  0
hour                     0
timeOfDay                0
mon                      0
offseason                0
DescLower               14
CapLength                0
popPlayer               14
popTeam                 14
at                      14
hash                    14
natlTV                  14
event                   14
dtype: int64

## modeling

In [106]:
# variables used in model, inclduing response variable
train_features = train[['Engagements','Type','weekend','offseason','timeOfDay','CapLength','popPlayer','popTeam','at','hash','natlTV','event']]

In [107]:
train_features.head()

,Engagements,Type,weekend,offseason,timeOfDay,CapLength,popPlayer,popTeam,at,hash,natlTV,event
0,502093,Video,N,N,Night,95.0,0,1,1,0,0,0
1,603380,Video,N,N,Night,64.0,0,1,1,0,1,0
2,603380,Video,N,N,Night,46.0,0,1,1,0,0,0
3,725100,Video,N,N,Night,43.0,1,0,0,0,1,0
4,661446,Video,N,N,Night,57.0,1,0,1,0,1,0


In [108]:
# one hot encode categorical variables and create dataframe used in modeling

# categorical columns
cat_columns = ['Type','weekend','offseason','timeOfDay','popPlayer','popTeam','at','hash','natlTV','event']
train_features_model = pd.get_dummies(data=train_features, columns=cat_columns)

train_features_model.head()

,Engagements,CapLength,Type_Album,Type_Photo,Type_Video,weekend_N,weekend_Y,offseason_N,offseason_Y,timeOfDay_Afternoon,...,popTeam_0,popTeam_1,at_0,at_1,hash_0,hash_1,natlTV_0,natlTV_1,event_0,event_1
0,502093,95.0,0,0,1,1,0,1,0,0,...,0,1,0,1,1,0,1,0,1,0
1,603380,64.0,0,0,1,1,0,1,0,0,...,0,1,0,1,1,0,0,1,1,0
2,603380,46.0,0,0,1,1,0,1,0,0,...,0,1,0,1,1,0,1,0,1,0
3,725100,43.0,0,0,1,1,0,1,0,0,...,1,0,1,0,1,0,0,1,1,0
4,661446,57.0,0,0,1,1,0,1,0,0,...,1,0,0,1,1,0,0,1,1,0


In [110]:
## create features and response data frames
features = train_features_model[train_features_model.columns.tolist()[1:]]
response = train_features_model['Engagements']

## cross validation shell..

In [111]:
# get CV indices
def getIndices(K):
    kf = KFold(n_splits=K, shuffle=True)
    train_ind = []
    test_ind = []

    for train_index, test_index in kf.split(train_features_model):
        train_ind.append(train_index)
        test_ind.append(test_index)
    return train_ind, test_ind

In [112]:
## MAPE function
## calculated for each "column" of predicted y values matrix corresponding to a different combo of params from CV
def calc_mape(y_pred,y_true):
    m = []
    for i in range(0,y_pred.shape[1]):
        mape = np.mean(np.abs((y_true - y_pred[:,i]) / y_true)) * 100
        m.append(mape)
    return m


### boosted tree tuning

In [123]:
## parameters
## initially tuning lambda = .001, .01, .1 (and adjusted ranges above and below .1 after initial results)
## initially tuning trees = 50, 100, 200 (and adjusted ranges above and below 100 after initial results)
## initially tuning max_depth = 2,3,4,5,6,7,8
lambdas = [.085,.09,.095]
n_tree = [105,110,115]
max_depth = [3,4]

np.random.seed(132) 
## number of folds and reps
K = 10
Nrep = 5
## number of models to be tested
n_models = len(lambdas)*len(n_tree)*len(max_depth)

## initialized empty matrices for MAPE, models (combo of parameters), and predicted y values
MAPE  = np.zeros((Nrep,n_models))
models = np.zeros((n_models,3))
y_pred_bs = np.zeros((train_features_model.shape[0],n_models))
## actual engagements
y_act = response.tolist()


## multiple reps of K-fold to ultimately populated MAPE matrix which will be Nrep x Nmodels dimension
for nr in range(0,Nrep):
    x = getIndices(K)
    for k in range(0,K):
        i = 0
        for l in lambdas:
            for t in n_tree:
                for m in max_depth:
                    models[i] = [l,t,m]
                    b = GradientBoostingRegressor(learning_rate=l, n_estimators=t, max_depth=m)
                    b.fit(features.iloc[x[0][k].tolist(),], response.iloc[x[0][k].tolist(),])
                    y_pred_bs[x[1][k].tolist(),i] = b.predict(features.iloc[x[1][k].tolist(),]).tolist()
                    i +=1
    MAPE[nr,:] = calc_mape(y_pred_bs,y_act)


In [124]:
## calculate the average CV MAPE per model tested
mape_total = MAPE.mean(0)
# find the best MAPE score
best_mape = min(mape_total)
## find the paramters corresponding to the min (best) MAPE
best_l = models[np.argmin(mape_total)][0]
best_t = int(models[np.argmin(mape_total)][1])
best_m = int(models[np.argmin(mape_total)][2])
# print the best model
print("Best Model:  " + "lambda=" + str(best_l) + ", ntrees=" + str(best_t) + ", max_depth=" + str(best_m) + ", MAPE=" + str(best_mape))

Best Model:  lambda=0.095, ntrees=105, max_depth=4, MAPE=9.324757584058904


In [80]:
## fit best boosted tree on data split into test train

In [125]:
X_train, X_test, y_train, y_test = train_test_split(features, response, test_size=0.2, random_state=132)

In [126]:
b = GradientBoostingRegressor(learning_rate=best_l, n_estimators=best_t, max_depth=best_m)
b.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.095, loss='ls', max_depth=4,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators=105,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [127]:
# predict on test set
pred = b.predict(X_test)

In [128]:
# calculate MAPE
np.mean(np.abs((y_test - pred) / y_test)) * 100

9.25709170404691

In [129]:
# feature importance
df = pd.DataFrame({'features':features.columns.tolist(),'impt':b.feature_importances_.tolist()})
df

,features,impt
0,CapLength,0.339958
1,Type_Album,0.005159
2,Type_Photo,0.011027
3,Type_Video,0.263596
4,weekend_N,0.019686
5,weekend_Y,0.015754
6,offseason_N,0.028462
7,offseason_Y,0.024007
8,timeOfDay_Afternoon,0.014368
9,timeOfDay_Evening,0.009077


### linear regression

In [121]:
K = 10
Nrep = 5
n_models = 1

MAPE  = np.zeros((Nrep,n_models))
y_pred_lr = np.zeros((train_features_model.shape[0],n_models))
y_act = response.tolist()



for nr in range(0,Nrep):
    x = getIndices(K)
    for k in range(0,K):
        i = 0
        
        lr = LinearRegression()
        lr.fit(features.iloc[x[0][k].tolist(),], response.iloc[x[0][k].tolist(),])
        y_pred_lr[x[1][k].tolist(),i] = lr.predict(features.iloc[x[1][k].tolist(),]).tolist()
        i +=1
    #fscore=score(y_test,y_pred_bs)
    MAPE[nr,:] = calc_mape(y_pred_lr,y_act)

In [122]:
np.mean(MAPE)

10.205976026950305